In [15]:
%pip install pandas
%pip install numpy
%pip install seaborn
%pip install matplotlib
%pip install six
%pip install sklearn
%pip install yellowbrick
%pip install pydotplus
%pip install graphviz
%pip install verta
%pip install Minio
%pip install category_encoders
# %pip install modin[ray]
%pip install boto3

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [41]:
import os
# os.environ["MODIN_ENGINE"] = "ray"


In [42]:
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
# import pandas as pd
# import modin.pandas as pd

import watermark
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime
import verta.integrations.sklearn
from minio import Minio
from verta import Client
from minio.error import ResponseError
import os
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline


# import tools as tools
%matplotlib inline
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [43]:
%watermark -n -v -m -g -iv


Python implementation: CPython
Python version       : 3.7.5
IPython version      : 7.16.1

Compiler    : Clang 11.0.3 (clang-1103.0.32.62)
OS          : Darwin
Release     : 20.2.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

Git hash: 2ddb6706e28be8b55a0456a8da905ef012b46a7e

seaborn          : 0.11.0
sys              : 3.7.5 (default, Jul  7 2020, 21:09:45) 
[Clang 11.0.3 (clang-1103.0.32.62)]
matplotlib       : 3.3.1
watermark        : 2.1.0
joblib           : 0.14.1
sklearn          : 0.0
pandas           : 1.1.4
category_encoders: 2.2.2
numpy            : 1.19.0



In [44]:
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d%Y%H%M%S%f")
experiment_name = "customerchurn"
experiment_id = experiment_name + timestampStr

def get_s3_server():
    minioClient = Minio('minio-ml-workshop:9000',
                    access_key='minio',
                    secret_key='minio123',
                    secure=False)

    return minioClient

def get_verta():
    client = Client("http://v1-webapp:3000")
    return client

def get_meta_store():
    client = get_verta()
    proj = client.set_project("ml-workshop")
    client.set_experiment(experiment_name)
    run = client.set_experiment_run(experiment_id)
    return run


# def record_metrics(classifier, expereiment_name, accuracy_score, hyperparameters):
#     client = Client("http://chart-1603715522-webapp:3000")
#     proj = client.set_project("HDFC DEmo PRoject")
#     client.set_experiment("Iris Classifier")
#     run = client.set_experiment_run(expereiment_name)
#
#     for key, value in hyperparameters.items():
#         run.log_hyperparameters({key : value})
#
#     run.log_metric('accuracy', accuracy_score)
#     run.log_tags([classifier])


In [45]:
minioClient = get_s3_server()
data_file = minioClient.fget_object("data", "full_data_csv/data.csv/part-00000-40d2e216-2992-4781-ba15-9789cb64ae30-c000.csv", "/tmp/data.csv")
data_file_version = data_file.version_id
data = pd.read_csv('/tmp/data.csv')
data.head(5)

# from verta.dataset import S3
# os.environ["AWS_ACCESS_KEY_ID"] = "minio"
# os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
#
# dataset = get_verta().get_dataset(name="MLWorkshop Churn Data")
# dataset_version = dataset.create_version(S3("s3://data/full_Data_csv/part-00000-56275f21-afd6-4049-a32a-e0252789a607-c000.csv"))

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [1]:
# %pip install boto3
# import boto3
# from verta.dataset import S3
# from verta.dataset import Path
# from boto3.client import Config
#
# # os.environ["AWS_ACCESS_KEY_ID"] = "minio"
# # os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
# # os.environ["endpoint_url"] = "minio-ml-workshop:9000"
#
# # s3 = boto3.resource('s3',
# #   endpoint_url='http://minio-ml-workshop:9000',
# #   config=boto3.session.Config(signature_version='s3v4')
# # )
#
# s3 = boto3.resource('s3',
#                     endpoint_url='http://minio-ml-workshop:9000',
#                     aws_access_key_id='minio',
#                     aws_secret_access_key='minio123',
#                     config=Config(signature_version='s3v4'),
#                     region_name='us-east-1')
# #
# dataset= get_verta('123').get_dataset(name="MLWorkshop Churn Data")
#
# dataset_version = dataset.create_version(  S3("s3://data/full_Data_csv/part-00000-56275f21-afd6-4049-a32a-e0252789a607-c000.csv"))
#


     |████████████████████████████████| 129 kB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 7.0 MB 11.9 MB/s eta 0:00:01
  Using cached s3transfer-0.3.3-py2.py3-none-any.whl (69 kB)
Note: you may need to restart the kernel to use updated packages.


NameError: name 'get_verta' is not defined

In [46]:
data.shape

(7043, 21)

In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [48]:
data.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [49]:
data.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [51]:
# Convert binary variable into numeric so plotting is easier. We need to later take mean
data['Churn'] = data['Churn'].map({'Yes': 1, 'No': 0})

In [53]:
data.replace(" ", np.nan, inplace=True)

In [54]:
data.isna().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [55]:
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'])

In [56]:
mean = data['TotalCharges'].mean()
data.fillna(mean, inplace=True)
# Now we know that total charges has nan values
data.isna().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

## Feature Engineering pipelne


In [59]:
import category_encoders as ce
import joblib

names = ['gender', 'Partner', 'Dependents', 'PhoneService', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn']
# for column in names:
#     labelencoder(column)

enc = ce.ordinal.OrdinalEncoder(cols=names)
enc.fit(data)
joblib.dump(enc, 'enc.pkl')
labelled_set = enc.transform(data)
labelled_set.tail(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
7038,6840-RESVB,2,0,1,2,24,2,Yes,DSL,Yes,...,Yes,Yes,2,2,One year,1,Mailed check,84.80,1990.50,1
7039,2234-XADUH,1,0,1,2,72,2,Yes,Fiber optic,No,...,Yes,No,2,2,One year,1,Credit card (automatic),103.20,7362.90,1
7040,4801-JZAZL,1,0,1,2,11,1,No phone service,DSL,Yes,...,No,No,1,1,Month-to-month,1,Electronic check,29.60,346.45,1
7041,8361-LTMKD,2,1,1,1,4,2,Yes,Fiber optic,No,...,No,No,1,1,Month-to-month,1,Mailed check,74.40,306.60,2
7042,3186-AJIEK,2,0,2,1,66,2,No,Fiber optic,Yes,...,Yes,Yes,2,2,Two year,1,Bank transfer (automatic),105.65,6844.50,1


In [66]:

names = ['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod', 'OnlineSecurity', 'OnlineBackup',
         'DeviceProtection', 'TechSupport']

ohe = ce.OneHotEncoder(cols=names)
ohe.fit(labelled_set)
joblib.dump(ohe, 'ohe.pkl')
final_set = ohe.transform(labelled_set)
final_set.tail(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines_1,MultipleLines_2,MultipleLines_3,...,Contract_2,Contract_3,PaperlessBilling,PaymentMethod_1,PaymentMethod_2,PaymentMethod_3,PaymentMethod_4,MonthlyCharges,TotalCharges,Churn
7038,6840-RESVB,2,0,1,2,24,2,0,0,1,...,1,0,1,0,1,0,0,84.80,1990.50,1
7039,2234-XADUH,1,0,1,2,72,2,0,0,1,...,1,0,1,0,0,0,1,103.20,7362.90,1
7040,4801-JZAZL,1,0,1,2,11,1,1,0,0,...,0,0,1,1,0,0,0,29.60,346.45,1
7041,8361-LTMKD,2,1,1,1,4,2,0,0,1,...,0,0,1,0,1,0,0,74.40,306.60,2
7042,3186-AJIEK,2,0,2,1,66,2,0,1,0,...,0,1,1,0,0,1,0,105.65,6844.50,1


In [69]:
labels = final_set['Churn']
X_train, X_test, y_train, y_test = train_test_split(final_set, labels, test_size=0.2)
X_train.pop('Churn')
X_train.pop('customerID')
X_test.pop('Churn')
X_test.pop('customerID')
print ('Training Data Shape',X_train.shape, y_train.shape)
print ('Testing Data Shape',X_test.shape, y_test.shape)

Training Data Shape (5634, 36) (5634,)
Testing Data Shape (1409, 36) (1409,)


In [70]:

# Data For cross validation and GridSearch
Y = final_set['Churn']
X = final_set.drop(['Churn', 'customerID'], axis=1)
print ('Training Data Shape', X.shape)
print ('Testing Data Shape', Y.shape)

Training Data Shape (7043, 36)
Testing Data Shape (7043,)


In [71]:
# Create decision tree object
DT = DecisionTreeClassifier()
# List of parameters
# entropy
criterion = ['gini']
max_depth = [5,10,15]
min_samples_split = [2,4,6]
min_samples_leaf = [4,5,6,8]
# Save all the lists in the variable
hyperparameters = dict(max_depth=max_depth, criterion=criterion,min_samples_leaf = min_samples_leaf ,min_samples_split = min_samples_split)

In [72]:
model = GridSearchCV(DT, hyperparameters, cv=5, verbose=0)
best_model = model.fit(X,Y)

In [73]:
# Mean cross validated score
print('Mean Cross-Validated Score: ',best_model.best_score_)
print('Best Parameters',best_model.best_params_)
# You can also print the best penalty and C value individually from best_model.best_estimator_.get_params()
print('Best criteria:', best_model.best_estimator_.get_params()['criterion'])
print('Best depth:', best_model.best_estimator_.get_params()['max_depth'])

Mean Cross-Validated Score:  0.7942641460739402
Best Parameters {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 6, 'min_samples_split': 2}
Best criteria: gini
Best depth: 5


In [74]:
kfold = KFold(n_splits = 3)
hyperparameters = dict(max_depth=5, criterion='gini',min_samples_leaf = 3 ,min_samples_split = 10)
model = DecisionTreeClassifier(max_depth=5, criterion='gini',min_samples_leaf = 3 ,min_samples_split = 10)
model = model.fit(X_train, y_train)
joblib.dump(model, 'dct.pkl')
results = model_selection.cross_val_score(model,X,Y,cv = kfold)
print(results)
print('Accuracy',results.mean()*100)
store = get_meta_store()
store.log_hyperparameters(hyperparameters)
store.log_model(model)
store.log_metric('Accuracy',results.mean()*100)
store.log_tag("DecisionTreeClassifier")
# get_meta_store().log_dataset_version("raw_data", dataset_version)

[0.79897785 0.7955707  0.79037069]
Accuracy 79.49730793137397


In [75]:
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d%Y%H%M%S%f")
experiment_name = "customerchurn"
experiment_id = experiment_name + timestampStr


# Create random forest object
RF = RandomForestClassifier()
n_estimators = [18,22]
criterion = ['gini', 'entropy']
# Create a list of all of the parameters
max_depth = [30,40,50]
min_samples_split = [6,8]
min_samples_leaf = [8,10,12]
# Merge the list into the variable
hyperparameters = dict(n_estimators = n_estimators,max_depth=max_depth, criterion=criterion,min_samples_leaf = min_samples_leaf ,min_samples_split = min_samples_split)
# Fit your model using gridsearch
model = GridSearchCV(RF, hyperparameters, cv=5, verbose=0)
best_model = model.fit(X, Y)

In [76]:
# Mean cross validated score
print('Mean Cross-Validated Score: ',best_model.best_score_)
print('Best Parameters',best_model.best_params_)
# You can also print the best penalty and C value individually from best_model.best_estimator_.get_params()
print('Best criteria:', best_model.best_estimator_.get_params()['criterion'])
print('Best depth:', best_model.best_estimator_.get_params()['max_depth'])
print('Best estimator:', best_model.best_estimator_.get_params()['n_estimators'])


Mean Cross-Validated Score:  0.805764888057294
Best Parameters {'criterion': 'entropy', 'max_depth': 50, 'min_samples_leaf': 10, 'min_samples_split': 8, 'n_estimators': 18}
Best criteria: entropy
Best depth: 50
Best estimator: 18


In [77]:
kfold = KFold(n_splits = 3)
hyperparameters = dict(max_depth=40, criterion='gini',min_samples_leaf = 12 ,min_samples_split = 8, n_estimators = 22)
model = RandomForestClassifier(max_depth=40, criterion='gini',min_samples_leaf = 12 ,min_samples_split = 8, n_estimators = 22)
model = model.fit(X_train, y_train)
joblib.dump(model, 'rft.pkl')
results = model_selection.cross_val_score(model,X,Y,cv = kfold)
print(results)
print('Accuracy',results.mean()*100)
store = get_meta_store()
store.log_hyperparameters(hyperparameters)
store.log_model(model)
store.log_metric('Accuracy',results.mean()*100)
store.log_tag("RandomForestClassifier")
store.log_attribute("data_file_location", "data/full_data_csv/a.csv")
store.log_attribute("data_file_version", data_file_version)

[0.81005111 0.79982964 0.79804005]
Accuracy 80.26402669010689
